# 최종 모델: Stacked GRU

### 1. Model 구성
- 베이스라인 모델 참조 Bidrirectional StackedGRU

### 2. 최종 모델 성능 최적화 방법
1. Model의 ouput에 lowpass filter를 적용해 noise를 제거 (데이터후처리)


### 3. 모델 선정 기준
1. 일반화 성능이 높은 모델 (Validation data TaPR과 Public score의 차이가 적은 모델)
2. Public score가 높은 모델 

### 4. 최종 모델 성능
- Publice score : 0.60163
- Private score : 0.60695

### 5. 시도해본 방법들

- 모델 관점 
    1. LSTM AutoEncoder 모델
    2. 1D CNN 모델

    
   
- 데이터 전처리관점
    1. feature selection
 

### 6. Path 정보

- Data Path
    1. traing data : data/train/
    2. validation data : data/validation/
    3. test data : data/test/
    
    
- Model Path
    1. model 저장 경로 : ./answer.pt
    2. 최종제출 model : ./answer.pt
    
    
- Submission Path
    1. submission 저장 경로 : ./
    2. sample path : data/sample_submission.csv
    3. 최종제출 submisstion : ./answer.csv
    
    
- TaPR Lib path
    1. whl file : data/eTaPR-21.8.2-py3-none-any.whl
    
### 7. Library 버전
- toolz==0.10.0
- torch==1.4.0
- torchvision==0.5.0
- torchviz==0.0.1
- tornado==6.0.4
- numpy==1.18.5
- notebook==6.0.3
- pyparsing==2.4.7
- python 3
- matplotlib 
- pandas
- dateutil
- datetime




In [ ]:
# !pip install data/eTaPR-21.8.2-py3-none-any.whl

In [ ]:
import sys
from pathlib import Path
from datetime import timedelta
import dateutil

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import torch
from torch.utils.data import Dataset, DataLoader
from tqdm.notebook import trange
from TaPR_pkg import etapr

#random seed를 사용함으로써 매번 달라지는 training set과 validation set을 고정시킴
import random
random_seed=1011
random.seed(random_seed)
np.random.seed(random_seed)
torch.manual_seed(random_seed)
torch.cuda.manual_seed(random_seed)
torch.cuda.manual_seed_all(random_seed) # if use multi-GPU
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

#window size 설정
WINDOW_SIZE = 30
WINDOW_GIVEN = WINDOW_SIZE-1

idx="answer"

## Utils

* lowpass filter, moving average, range check, gray area 함수를 저장합니다.

In [ ]:
# 모델의 결과(anomaly score)를 smoothing하기 위해서 다양한 filter를 적용
# 데이터 후처리(lowpass필터 적용)를 진행하면 아래와 같이 anomaly score가 더 안정적으로 변함을 확인할 수 있다. (Test Dataset에서는 그 효과가 더 자세히 나타난다.)
# the order of the filer : 1
# the ciritical frequency of the filter : 0.1
from scipy import signal
b, a = signal.butter(1, 0.1, btype='lowpass')


#Moving average
def moving_average(ANOMALY_SCORE, window=10):
    ma=[]
    for idx in range(len(ANOMALY_SCORE)):
        if idx >= window:
            ma.append((ANOMALY_SCORE[idx-window:idx].mean()+ANOMALY_SCORE[idx])/2)
        else:
            ma.append(ANOMALY_SCORE[idx])
    ma=np.array(ma)
    return ma


#Range check
def range_check(series, size):
    data = []
    for i in range(len(series)-size+1):
        if i == 0 :
            check_std = np.std(series[i:i+size])
        std = np.std(series[i:i+size])
        mean = np.mean(series[i:i+size])
        max = np.max(series[i:i+size])
        if check_std * 2 >= std:
            data.append(mean)
            check_std = std
        elif max == series[i]:
            data.append(max*5)
            check_std = std
        else:
            data.append(series[i]*3)
    for _ in range(size-1):
        data.append(mean)
    return np.array(data)


#Gray Area(2020 수상팀인 SIILAB의 Gray Aare Smoothing 코드  참조)
def Gray_Area(attacks):
    start = []  # start point
    finish = []  # finish point
    c = []  # count
    com = 0
    count = 0
    for i in range(1, len(attacks)):
        if attacks[i - 1] != attacks[i]:
            if com == 0:
                start.append(i)
                count = count + 1
                com = 1
            elif com == 1:
                finish.append(i - 1)
                c.append(count)
                count = 0
                start.append(i)
                count = count + 1
        else:
            count = count + 1
    finish.append(len(attacks) - 1)
    c.append(finish[len(finish) - 1] - start[len(start) - 1] + 1)
    for i in range(0, len(start)):
        if c[i] < 10:
            s = start[i]
            f = finish[i] + 1
            g1 = [1 for i in range(c[i])] # Temp Attack list
            g0 = [0 for i in range(c[i])]  # Temp Normal List
            if attacks[start[i] - 1] == 1:
                attacks[s:f] = g1  # change to attack
            else:
                attacks[s:f] = g0  # change to normal
    return attacks

## Preprocessing

* Train data, Test data, Validation data 에 대해 csv 데이터를 읽어오고 합치는 과정을 수행합니다. timpestamp 필드만 삭제하고 모델을 학습하였습니다. min-max scaler를 사용하여 feature scaling을 진행하였습니다. 그 후 ewm(alpha=0.9)를 적용해 노이즈를 제거하였습니다.

In [ ]:
def dataframe_from_csv(target):
    return pd.read_csv(target).rename(columns=lambda x: x.strip())

def dataframe_from_csvs(targets):
    return pd.concat([dataframe_from_csv(x) for x in targets])

TRAIN_DATASET = sorted([x for x in Path("data/train/").glob("*.csv")])
VALIDATION_DATASET = sorted([x for x in Path("data/validation/").glob("*.csv")])
TEST_DATASET = sorted([x for x in Path("data/test/").glob("*.csv")])

TRAIN_DF_RAW = dataframe_from_csvs(TRAIN_DATASET)
VALIDATION_DF_RAW = dataframe_from_csvs(VALIDATION_DATASET)
TEST_DF_RAW = dataframe_from_csvs(TEST_DATASET)

TIMESTAMP_FIELD = "timestamp"
ATTACK_FIELD = "attack"
VALID_COLUMNS_IN_TRAIN_DATASET = TRAIN_DF_RAW.columns.drop([TIMESTAMP_FIELD])

TAG_MIN = TRAIN_DF_RAW[VALID_COLUMNS_IN_TRAIN_DATASET].min()
TAG_MAX = TRAIN_DF_RAW[VALID_COLUMNS_IN_TRAIN_DATASET].max()

#Normalization
def normalize(df):
    ndf = df.copy()
    for c in df.columns:
        if TAG_MIN[c] == TAG_MAX[c]:
            ndf[c] = df[c] - TAG_MIN[c]
        else:
            ndf[c] = (df[c] - TAG_MIN[c]) / (TAG_MAX[c] - TAG_MIN[c])
    return ndf

TRAIN_DF = normalize(TRAIN_DF_RAW[VALID_COLUMNS_IN_TRAIN_DATASET]).ewm(alpha=0.9).mean()
VALIDATION_DF = normalize(VALIDATION_DF_RAW[VALID_COLUMNS_IN_TRAIN_DATASET])
TEST_DF = normalize(TEST_DF_RAW[VALID_COLUMNS_IN_TRAIN_DATASET]).ewm(alpha=0.9).mean()

#boundary check
def boundary_check(df):
    x = np.array(df, dtype=np.float32)
    return np.any(x > 1.0), np.any(x < 0), np.any(np.isnan(x))

boundary_check(TRAIN_DF), boundary_check(VALIDATION_DF), boundary_check(TEST_DF)

## Dataset load

In [ ]:
class HaiDataset(Dataset):
    def __init__(self, timestamps, df, stride=1, attacks=None):
        self.ts = np.array(timestamps)
        self.tag_values = np.array(df, dtype=np.float32)
        self.valid_idxs = []
        for L in trange(len(self.ts) - WINDOW_SIZE + 1):
            R = L + WINDOW_SIZE - 1
            if dateutil.parser.parse(self.ts[R]) - dateutil.parser.parse(
                self.ts[L]
            ) == timedelta(seconds=WINDOW_SIZE - 1):
                self.valid_idxs.append(L)
        self.valid_idxs = np.array(self.valid_idxs, dtype=np.int32)[::stride]
        self.n_idxs = len(self.valid_idxs)
        print(f"# of valid windows: {self.n_idxs}")
        if attacks is not None:
            self.attacks = np.array(attacks, dtype=np.float32)
            self.with_attack = True
        else:
            self.with_attack = False

    def __len__(self):
        return self.n_idxs

    def __getitem__(self, idx):
        i = self.valid_idxs[idx]
        last = i + WINDOW_SIZE - 1
        item = {"attack": self.attacks[last]} if self.with_attack else {}
        item["ts"] = self.ts[i + WINDOW_SIZE - 1]
        item["given"] = torch.from_numpy(self.tag_values[i : i + WINDOW_GIVEN])
        item["answer"] = torch.from_numpy(self.tag_values[last])
        return item
    
HAI_DATASET_TRAIN = HaiDataset(TRAIN_DF_RAW[TIMESTAMP_FIELD], TRAIN_DF, stride=1)
HAI_DATASET_VALIDATION = HaiDataset(
    VALIDATION_DF_RAW[TIMESTAMP_FIELD], VALIDATION_DF, attacks=VALIDATION_DF_RAW[ATTACK_FIELD]
)
HAI_DATASET_TEST = HaiDataset(
    TEST_DF_RAW[TIMESTAMP_FIELD], TEST_DF, attacks=None
)

## Modeling

* 모델 하이퍼파라미터 설정입니다.  
    * Hidden cells : 200  
    * Layer : 3  
    * Batch size : 4096  
    * Epoch : 200  
    * Drop out : 0    
    * Learning rate : 1e-3  
    * Moving average는 window size의 2배, range check는 30으로 설정하였습니다.
   


In [ ]:
import easydict
args = easydict.EasyDict({
                        'N_HIDDENS' : 200,
                        'N_LAYERS' : 3,
                        'batch_size' : 4096,
                        'epochs' : 200,
                        'dropout' : 0,
                        'lr' : 1e-3,
                        'moving_average' : WINDOW_SIZE*2,
                        'range_check' : 30
                        })

In [ ]:
class StackedGRU(torch.nn.Module):
    def __init__(self, n_tags):
        super().__init__()
        self.rnn = torch.nn.GRU(
            input_size=n_tags,
            hidden_size=args.N_HIDDENS,
            num_layers=args.N_LAYERS,
            bidirectional=True,
            dropout=args.dropout,
        )
        self.fc = torch.nn.Linear(args.N_HIDDENS * 2, n_tags)
        
    def forward(self, x):
        x = x.transpose(0, 1)  # (batch, seq, params) -> (seq, batch, params)
        self.rnn.flatten_parameters()
        outs, _ = self.rnn(x)
        out = self.fc(outs[-1])
        return x[0] + out
    
MODEL = StackedGRU(n_tags=TRAIN_DF.shape[1])
MODEL.cuda()

## Training

In [ ]:
%%time
def train(dataset, model, batch_size, n_epochs):
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
    optimizer = torch.optim.AdamW(model.parameters(), lr=args.lr)
    loss_fn = torch.nn.MSELoss()
    epochs = trange(n_epochs, desc="training")
    best = {"loss": sys.float_info.max}
    loss_history = []
    for e in epochs:
        epoch_loss = 0
        for batch in dataloader:
            optimizer.zero_grad()
            given = batch["given"].cuda()
            guess = model(given)
            answer = batch["answer"].cuda()
            loss = loss_fn(answer, guess)
            loss.backward()
            epoch_loss += loss.item()
            optimizer.step()
        loss_history.append(epoch_loss)
        epochs.set_postfix_str(f"loss: {epoch_loss:.6f}")
        if epoch_loss < best["loss"]:
            best["state"] = model.state_dict()
            best["loss"] = epoch_loss
            best["epoch"] = e + 1
    return best, loss_history

MODEL.train()
BEST_MODEL, LOSS_HISTORY = train(HAI_DATASET_TRAIN, MODEL, args.batch_size, args.epochs)
print(f'best_loss: {BEST_MODEL["loss"]}, best_epoch: {BEST_MODEL["epoch"]}.')

with open(idx+'.pt', "wb") as f:
    torch.save(
        {
            "state": BEST_MODEL["state"],
            "best_epoch": BEST_MODEL["epoch"],
            "loss_history": LOSS_HISTORY,
        },
        f,
    )

## Load model

In [ ]:
with open(idx+'.pt', "rb") as f:
    SAVED_MODEL = torch.load(f)

MODEL.load_state_dict(SAVED_MODEL["state"])

plt.figure(figsize=(16, 4))
plt.title("Training Loss Graph")
plt.xlabel("epochs")
plt.ylabel("loss")
plt.yscale("log")
plt.plot(SAVED_MODEL["loss_history"])
plt.show()

## Validation

In [ ]:
def inference(dataset, model, batch_size):
    dataloader = DataLoader(dataset, batch_size=batch_size)
    ts, dist, att = [], [], []
    with torch.no_grad():
        for batch in dataloader:
            given = batch["given"].cuda()
            answer = batch["answer"].cuda()
            guess = model(given)
            ts.append(np.array(batch["ts"]))
            dist.append(torch.abs(answer - guess).cpu().numpy())
            try:
                att.append(np.array(batch["attack"]))
            except:
                att.append(np.zeros(batch_size))
    return (
        np.concatenate(ts),
        np.concatenate(dist),
        np.concatenate(att),
    )

def check_graph(xs, att, piece=2, THRESHOLD=None):
    l = xs.shape[0]
    chunk = l // piece
    fig, axs = plt.subplots(piece, figsize=(20, 4 * piece))
    for i in range(piece):
        L = i * chunk
        R = min(L + chunk, l)
        xticks = range(L, R)
        axs[i].plot(xticks, xs[L:R])
        if len(xs[L:R]) > 0:
            peak = max(xs[L:R])
            axs[i].plot(xticks, att[L:R] * peak * 0.3)
        if THRESHOLD!=None:
            axs[i].axhline(y=THRESHOLD, color='r')
    plt.show()

def put_labels(distance, threshold):
    xs = np.zeros_like(distance)
    xs[distance > threshold] = 1
    return xs

def fill_blank(check_ts, labels, total_ts):
    def ts_generator():
        for t in total_ts:
            yield dateutil.parser.parse(t)
    def label_generator():
        for t, label in zip(check_ts, labels):
            yield dateutil.parser.parse(t), label
    g_ts = ts_generator()
    g_label = label_generator()
    final_labels = []
    try:
        current = next(g_ts)
        ts_label, label = next(g_label)
        while True:
            if current > ts_label:
                ts_label, label = next(g_label)
                continue
            elif current < ts_label:
                final_labels.append(0)
                current = next(g_ts)
                continue
            final_labels.append(label)
            current = next(g_ts)
            ts_label, label = next(g_label)
    except StopIteration:
        return np.array(final_labels, dtype=np.int8)

    
#threshold 설정 --> delta 값을 기준으로 최적의 threshold 값 선정    
def opt_thresh(anom_score, check_ts, total_ts, atk_label):
    # initial values
    tap, tar = 0.0, 1.0     # initial recall and precision
    q, d = 0.99, 0.01       # initial quantile and distance
    delta = 0.015
    # loop until tap and tar are close enough
    while abs(tap - tar) > delta:
        t, d = np.quantile(anom_score, q), d/2      # update t and d
        anom_label = fill_blank(check_ts, put_labels(anom_score, t), total_ts)
        tapr = etapr.evaluate_haicon(anomalies=atk_label, predictions=anom_label)
        tap, tar = float(tapr['TaP']), float(tapr['TaR'])
        if tar > tap:
            q = q + d
        else:
            q = q - d
    return t, tapr, anom_label

## 데이터 후처리 및 threshold 선정

In [ ]:
%%time

MODEL.eval()
CHECK_TS, CHECK_DIST, CHECK_ATT = inference(HAI_DATASET_VALIDATION, MODEL, args.batch_size)

ANOMALY_SCORE = np.mean(CHECK_DIST, axis=1)

ANOMALY_SCORE = signal.filtfilt(b,a,ANOMALY_SCORE)
ANOMALY_SCORE = moving_average(ANOMALY_SCORE, window=args.moving_average)
ANOMALY_SCORE = range_check(ANOMALY_SCORE, size=args.range_check)

thr, _, _ = opt_thresh(ANOMALY_SCORE, CHECK_TS, np.array(VALIDATION_DF_RAW[TIMESTAMP_FIELD]), CHECK_ATT)
print('threshold:', thr)
THRESHOLD = 0.008

LABELS = put_labels(ANOMALY_SCORE, THRESHOLD)
ATTACK_LABELS = put_labels(np.array(VALIDATION_DF_RAW[ATTACK_FIELD]), threshold=0.5)
FINAL_LABELS = fill_blank(CHECK_TS, LABELS, np.array(VALIDATION_DF_RAW[TIMESTAMP_FIELD]))
FINAL_LABELS = Gray_Area(LABELS)

check_graph(ANOMALY_SCORE, CHECK_ATT, piece=2, THRESHOLD=THRESHOLD)

TaPR = etapr.evaluate_haicon(anomalies=ATTACK_LABELS, predictions=FINAL_LABELS)
print(f"F1: {TaPR['f1']:.3f} (TaP: {TaPR['TaP']:.3f}, TaR: {TaPR['TaR']:.3f})")
print(f"# of detected anomalies: {len(TaPR['Detected_Anomalies'])}")
print(f"Detected anomalies: {TaPR['Detected_Anomalies']}")

## Test

In [ ]:
%%time
MODEL.eval()
CHECK_TS, CHECK_DIST, CHECK_ATT = inference(HAI_DATASET_TEST, MODEL, args.batch_size)

ANOMALY_SCORE = np.mean(CHECK_DIST, axis=1)

ANOMALY_SCORE = signal.filtfilt(b,a,ANOMALY_SCORE)
ANOMALY_SCORE = moving_average(ANOMALY_SCORE, window=args.moving_average)
ANOMALY_SCORE = range_check(ANOMALY_SCORE, size=args.range_check)

LABELS = put_labels(ANOMALY_SCORE, THRESHOLD)

check_graph(ANOMALY_SCORE, CHECK_ATT, piece=3, THRESHOLD=THRESHOLD)

## Submission

In [ ]:
submission = pd.read_csv('data/sample_submission.csv')
submission.index = submission['timestamp']
submission.loc[CHECK_TS,'attack'] = LABELS
submission['attack']=Gray_Area(submission['attack'].copy())
submission.to_csv(idx+'.csv', index=False)

submission['attack'].value_counts()